In [1]:
# Preparação dos dados
import os

import numpy as np
import pandas as pd

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score

sns.set_theme()

In [2]:
## Importar dataset teste

data_path = '../data/' if os.path.exists(
    '../data/') else 'https://raw.githubusercontent.com/kreativermario/Projeto-DECD/master/data/'

test_path = data_path + 'treated/prepared/numeric/no-dates/dataset-numeric-high-tensions-test-no-dates.csv'

test_df = pd.read_csv(test_path)

In [3]:
## Importar dataset treino

data_path = '../data/' if os.path.exists(
    '../data/') else 'https://raw.githubusercontent.com/kreativermario/Projeto-DECD/master/data/'

train_path = data_path + 'treated/prepared/numeric/no-dates/dataset-numeric-high-tensions-train-no-dates.csv'

train_df = pd.read_csv(train_path)

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4641 entries, 0 to 4640
Data columns (total 5 columns):
 #   Column                              Non-Null Count  Dtype
---  ------                              --------------  -----
 0   energia_ativa_alta_tensao_kwh       4641 non-null   int64
 1   cpes_domestico_alta_tensao          4641 non-null   int64
 2   cpes_nao_domestico_alta_tensao      4641 non-null   int64
 3   cpes_outros_alta_tensao             4641 non-null   int64
 4   densidade_populacional_pessoas_km2  4641 non-null   int64
dtypes: int64(5)
memory usage: 181.4 KB


In [5]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273 entries, 0 to 272
Data columns (total 5 columns):
 #   Column                              Non-Null Count  Dtype
---  ------                              --------------  -----
 0   energia_ativa_alta_tensao_kwh       273 non-null    int64
 1   cpes_domestico_alta_tensao          273 non-null    int64
 2   cpes_nao_domestico_alta_tensao      273 non-null    int64
 3   cpes_outros_alta_tensao             273 non-null    int64
 4   densidade_populacional_pessoas_km2  273 non-null    int64
dtypes: int64(5)
memory usage: 10.8 KB


# Algoritmos de Aprendizagem supervisionada

In [6]:
regressors = {
    'Linear Regression': LinearRegression(),
    'k-NN': KNeighborsRegressor(n_neighbors=5),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest': RandomForestRegressor(n_estimators=100),
    'SVM': SVR(kernel='linear', max_iter=10000),
    'MLP': MLPRegressor(hidden_layer_sizes=(8,4), max_iter=20000)
}

In [7]:
cv_scores = pd.Series({
    name: np.mean(cross_val_score(regressor, train_df.drop(columns=['energia_ativa_alta_tensao_kwh']), train_df['energia_ativa_alta_tensao_kwh'], cv=5))
    for name, regressor in regressors.items()
})

cv_scores

Linear Regression    0.525342
k-NN                -0.053549
Decision Tree        0.042950
Random Forest        0.325033
SVM                  0.469171
MLP                  0.550805
dtype: float64

In [8]:
best = cv_scores.sort_values(ascending=False).index[0]
best_regressor = regressors[best]
best_regressor.fit(train_df.drop(columns=['energia_ativa_alta_tensao_kwh']), train_df['energia_ativa_alta_tensao_kwh'])

print(f'### {best} ###')

y_pred = best_regressor.predict(test_df.drop(columns=['energia_ativa_alta_tensao_kwh']))

print('r2: {}'.format(r2_score(test_df['energia_ativa_alta_tensao_kwh'], y_pred)))
print('mse: {}'.format(mean_squared_error(test_df['energia_ativa_alta_tensao_kwh'], y_pred)))
print('mae: {}'.format(mean_absolute_error(test_df['energia_ativa_alta_tensao_kwh'], y_pred)))

### MLP ###
r2: -12.477315763976
mse: 102401995022039.36
mae: 6532161.613770501


## Ordene os exemplos do conjunto de teste por ordem decrescente do erro da previsão do regressor e verifique se existe algum padrão relevante. 

In [9]:
df_error = test_df.copy()
df_error['error'] = np.abs(df_error['energia_ativa_alta_tensao_kwh'] - y_pred)

df_error.sort_values('error', ascending=False).head(20)

,energia_ativa_alta_tensao_kwh,cpes_domestico_alta_tensao,cpes_nao_domestico_alta_tensao,cpes_outros_alta_tensao,densidade_populacional_pessoas_km2,error
109,25962882,2,1226,0,5466,7.852140e+07
102,12592033,0,581,0,649,3.363624e+07
186,16932236,1,519,0,5753,3.302650e+07
253,7515787,0,465,0,1821,3.167408e+07
216,8451642,1,486,0,1215,3.142082e+07
200,9633627,1,494,0,639,2.989482e+07
108,9662301,0,490,0,231,2.880430e+07
251,10364095,1,482,0,667,2.828007e+07
39,5880938,0,380,0,308,2.417874e+07
111,7406105,1,366,0,1215,2.315446e+07
